# 標準正規分布表を使った確率計算（インタラクティブ）

標準正規分布表（Z表）の読み方を、スライダーで動かしながら確認できるページです。
スライダーで z 値を動かすと、
- 左側のグラフで z 以下の面積がハイライトされる
- 表の該当セルが強調表示される
という連動が起きます。


## 例題

\( Z \sim \mathcal{N}(0, 1) \) のとき \( P(Z \le 1.23) \) を求める。

このとき z = 1.23 に対応する表の行（1.2）と列（0.03）を見れば、
\( P(Z \le 1.23) \) が読み取れます。


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, VBox, Output
from IPython.display import display
from math import erf

def normal_cdf(z):
    return 0.5 * (1 + erf(z / np.sqrt(2)))

rows = np.round(np.arange(0.0, 3.1, 0.1), 1)
cols = np.round(np.arange(0.00, 0.10, 0.01), 2)
table = pd.DataFrame(
    [[normal_cdf(r + c) for c in cols] for r in rows],
    index=rows,
    columns=cols,
)

output = Output()

def highlight_table(data, z_value):
    row = np.floor(z_value * 10) / 10
    col = np.round(z_value - row, 2)
    style = pd.DataFrame('', index=table.index, columns=table.columns)
    if row in style.index and col in style.columns:
        style.loc[row, col] = 'background-color: #ffe08a; color: #000; font-weight: bold;'
    return style

def update(z_value):
    with output:
        output.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(6, 3))
        xs = np.linspace(-3.5, 3.5, 400)
        ys = (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * xs**2)
        ax.plot(xs, ys, color='#1f77b4')
        xs_fill = xs[xs <= z_value]
        ys_fill = (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * xs_fill**2)
        ax.fill_between(xs_fill, ys_fill, color='#1f77b4', alpha=0.3)
        ax.axvline(z_value, color='#ff7f0e', linestyle='--')
        ax.set_title(f'z = {z_value:.2f},  P(Z ≤ z) = {normal_cdf(z_value):.4f}')
        ax.set_xlim(-3.5, 3.5)
        ax.set_ylim(0, 0.45)
        ax.set_xlabel('z')
        ax.set_ylabel('密度')
        plt.show()
        display(table.style.format('{:.4f}').apply(highlight_table, axis=None, z_value=z_value))

slider = FloatSlider(
    value=1.23,
    min=0.0,
    max=3.09,
    step=0.01,
    description='z',
    continuous_update=True,
)

def on_change(change):
    update(change['new'])

slider.observe(on_change, names='value')
display(VBox([slider, output]))
update(slider.value)


## 補足

- 表は \( z \ge 0 \) の範囲（0.00〜3.09）をカバーしています。
- \( z < 0 \) の場合は対称性 \( P(Z \le -z) = 1 - P(Z \le z) \) を利用します。
